# Investigate grids and sub grids for generating a cumaliative vtk file of all relevant quanitities

Creating grids for each quantiy individually leads to non-overlapping gridpoints due to floating point roundoffs.
1. Create global grid at the maximum extent of all quanitites.
2. Create subgrids by using np.argwhere(np.logical_and.reduce()) with a list of spatial limits.
3. use the return from argwhere as the interpolation gridpoints for girddata
4. then use swapaxes and reshape the make a vtkgrid and use it to subindex and fill a zeros array of the shape of the global grid.


In [1]:
import numpy as np
from scipy.interpolate import griddata

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

import sys

In [2]:
from pyvisfile.vtk import write_structured_grid
from pytools.obj_array import make_obj_array

In [3]:
sys.path.append('../../read_from_sql/')
import read_from_sql
sys.path.append('/Users/vonderlinden2/rsx_analysis/mach_probe_analysis')
sys.path.append('/Users/vonderlinden2/rsx_analysis/time_alignment/source/')
import ion_current_to_mach_number as ic_to_mach
reload(ic_to_mach)
sys.path.append('/Users/vonderlinden2/rsx_analysis/time_alignment/source')
import absolute_times as at

import structured_3d_vtk as struc_3d
reload(struc_3d)

<module 'structured_3d_vtk' from 'structured_3d_vtk.pyc'>

In [4]:
spatial_increment = 0.001
x_min, x_max = -0.027, 0.024
y_min, y_max = -0.021, 0.03
z_min, z_max = 0.249, 0.416
full_grid_bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

full_grid, sizes = struc_3d.bounded_grid(full_grid_bounds, spatial_increment)
full_vtk_grid = struc_3d.prepare_mesh(full_grid, sizes)

In [40]:
tp_vtk_grid_indices = np.where(np.logical_and.reduce([full_vtk_grid[0] >= -0.022,
                                                         full_vtk_grid[0] <= 0.018,
                                                         full_vtk_grid[1] >= -0.021, 
                                                         full_vtk_grid[1] <= 0.0255, 
                                                         full_vtk_grid[2] >= 0.249,
                                                         full_vtk_grid[2] <= 0.416]))

In [41]:
full_vtk_grid[0][tp_vtk_grid_indices[0], tp_vtk_grid_indices[1], tp_vtk_grid_indices[2]]

array([-0.0219 , -0.0219 , -0.0219 , ...,  0.01788,  0.01788,  0.01788])

In [37]:
tp_vtk_grid_indices5 = np.argwhere(np.logical_and.reduce([full_vtk_grid[0] >= -0.022,
                                                         full_vtk_grid[0] <= 0.018,
                                                         full_vtk_grid[1] >= -0.021, 
                                                         full_vtk_grid[1] <= 0.0255, 
                                                         full_vtk_grid[2] >= 0.249,
                                                         full_vtk_grid[2] <= 0.416]))

In [38]:
full_vtk_grid[0][tp_vtk_grid_indices5[:,0], tp_vtk_grid_indices5[:,1], tp_vtk_grid_indices5[:,2]]

array([-0.0219 , -0.0219 , -0.0219 , ...,  0.01788,  0.01788,  0.01788])

In [ ]:
full_vtk_grid[0][tp_vtk_grid_indices]

In [6]:
full_vtk_grid.shape

(3, 51, 51, 167)

In [7]:
51*51*167

434367

In [8]:
tp_vtk_grid_indices[2].shape

(307280,)

In [9]:
print tp_vtk_grid_indices

(array([ 5,  5,  5, ..., 44, 44, 44]), array([ 0,  0,  0, ..., 45, 45, 45]), array([  0,   1,   2, ..., 164, 165, 166]))


In [10]:
np.unique(tp_vtk_grid_indices[2]).size

167

In [11]:
np.unique(tp_vtk_grid_indices[0]).size

40

In [12]:
np.unique(tp_vtk_grid_indices[1]).size

46

In [13]:
167*40*46

307280

In [14]:
tp_0 = np.reshape(tp_vtk_grid_indices[0], (40, 46, 167))
tp_1 = np.reshape(tp_vtk_grid_indices[1], (40, 46, 167))
tp_2 = np.reshape(tp_vtk_grid_indices[2], (40, 46, 167))

In [15]:
full_vtk_grid[0][tp_0, tp_1, tp_2].shape

(40, 46, 167)

In [16]:
full_vtk_grid[0][tp_0, tp_1, tp_2]

array([[[-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        ..., 
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ]],

       [[-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        ..., 
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088]],

       [[-0.01986, -0.01986, -0.01986, ..., -0.01986, -0.01986, -0.0

In [17]:
tp_vtk_grid_indices2 = np.argwhere(np.logical_and.reduce([full_vtk_grid[0] >= -0.022,
                                                         full_vtk_grid[0] <= 0.018,
                                                         full_vtk_grid[1] >= -0.021, 
                                                         full_vtk_grid[1] <= 0.0255, 
                                                         full_vtk_grid[2] >= 0.249,
                                                         full_vtk_grid[2] <= 0.416]))
tp_vtk_grid_indices2.shape

(307280, 3)

In [18]:
tp_vtk_grid_indices3 = np.swapaxes(tp_vtk_grid_indices2, 0, 1)

tp_all = np.reshape(tp_vtk_grid_indices3, (3, 40, 46, 167))

tp_0 = np.reshape(tp_vtk_grid_indices3[0], (40, 46, 167))
tp_1 = np.reshape(tp_vtk_grid_indices3[1], (40, 46, 167))
tp_2 = np.reshape(tp_vtk_grid_indices3[2], (40, 46, 167))

In [20]:
full_vtk_grid[0][tp_all[0], tp_all[1], tp_all[2]]

array([[[-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        ..., 
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ],
        [-0.0219 , -0.0219 , -0.0219 , ..., -0.0219 , -0.0219 , -0.0219 ]],

       [[-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        ..., 
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088],
        [-0.02088, -0.02088, -0.02088, ..., -0.02088, -0.02088, -0.02088]],

       [[-0.01986, -0.01986, -0.01986, ..., -0.01986, -0.01986, -0.0

In [23]:
np.sum(np.invert(full_vtk_grid[0][tp_0, tp_1, tp_2] == full_vtk_grid[0][tp_all[0], tp_all[1], tp_all[2]]))

0